<a href="https://colab.research.google.com/github/nabilafirdaiss/DatasetNarkotika_2022-446_2022-455/blob/main/colab_tkiuts_2022_446_2022_455.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import os
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
stop_words = stopwords.words('indonesian')

In [8]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/dataset/Overview.xlsx'
data = pd.read_excel(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Head Data

In [29]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')

display(data.head())

,No,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan,Gabungan_Teks
0,1,132/Pid.Sus/2025/PN,PN Banda Aceh,3 tiga bungkusan plastik warna bening yang ber...,menyatakan terdakwa muhammad nadid bin sulaima...,3 tiga bungkusan plastik warna bening yang ber...
1,2,133/Pid.Sus/2025/PN,PN Banda Aceh,3 tiga bungkusan plastik warna bening yang ber...,menyatakan terdakwa zulfahmi alias bom bom bin...,3 tiga bungkusan plastik warna bening yang ber...
2,3,130/Pid.Sus/2025/PN,PN Banda Aceh,1 satu bungkusan plastik bening yang berisikan...,menyatakan terdakwa dedek bin sumadi terbukti ...,1 satu bungkusan plastik bening yang berisikan...
3,4,129/Pid.Sus/2025/PN,PN Banda Aceh,1 satu bungkusan plastik bening yang berisikan...,menyatakan terdakwa ridwan bin almarhum muhamm...,1 satu bungkusan plastik bening yang berisikan...
4,5,135/Pid.Sus/2025/PN,PN Banda Aceh,1 satu bungkusan plastic warna bening yang ber...,menyatakan terdakwa junawan bin suharto terbuk...,1 satu bungkusan plastic warna bening yang ber...


## Preprocessing

Penyesuaian nama Kolom  dataset


In [30]:
print(data.columns.tolist())
data.columns = data.columns.str.strip()
print("Sesudah strip:", data.columns.tolist())


['No', 'No Putusan', 'Lembaga Peradilan', 'Barang Bukti', 'Amar Putusan', 'Gabungan_Teks']
Sesudah strip: ['No', 'No Putusan', 'Lembaga Peradilan', 'Barang Bukti', 'Amar Putusan', 'Gabungan_Teks']


Hapus baris duplikat dan kosong

In [31]:
data.drop_duplicates(inplace=True)
data.dropna(subset=['Barang Bukti', 'Amar Putusan'], inplace=True)

Rapikan nama kolom (hapus spasi di nama kolom)

In [32]:
data.columns = data.columns.str.strip()

Cleaning

In [33]:
def clean_text(text):
    text = str(text).lower()                      # ubah ke huruf kecil
    text = re.sub(r'[^a-z0-9\s.,]', '', text)     # hapus karakter aneh
    text = re.sub(r'\s+', ' ', text).strip()      # hilangkan spasi berlebih
    return text
data['Barang Bukti'] = data['Barang Bukti'].apply(clean_text)
data['Amar Putusan'] = data['Amar Putusan'].apply(clean_text)

In [34]:
data['Gabungan_Teks'] = data['Barang Bukti'] + ' ' + data['Amar Putusan']

display(data[['No', 'Lembaga Peradilan', 'Gabungan_Teks']].head())


,No,Lembaga Peradilan,Gabungan_Teks
0,1,PN Banda Aceh,3 tiga bungkusan plastik warna bening yang ber...
1,2,PN Banda Aceh,3 tiga bungkusan plastik warna bening yang ber...
2,3,PN Banda Aceh,1 satu bungkusan plastik bening yang berisikan...
3,4,PN Banda Aceh,1 satu bungkusan plastik bening yang berisikan...
4,5,PN Banda Aceh,1 satu bungkusan plastic warna bening yang ber...


## Indexing (TF-IDF)


In [37]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=5000)
tfidf_matrix = vectorizer.fit_transform(data['Gabungan_Teks'])

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


##Eksekusi

Pencarian, Kemiripan, dan Pemeringkatan

menggunakan query contoh: "jual beli sabu" untuk menguji sistem.

In [39]:
query_text = "jual beli sabu"
print(f"Query yang diuji: '{query_text}'")
print("-" * 30)

Query yang diuji: 'jual beli sabu'
------------------------------


Pengolahan Query

In [40]:
processed_query = clean_text(query_text)
print(f"Query setelah dibersihkan: '{processed_query}'")

Query setelah dibersihkan: 'jual beli sabu'


Vektorisasi Query

In [41]:
query_vector = vectorizer.transform([processed_query])
print(f"Bentuk Vektor Query: {query_vector.shape}")

Bentuk Vektor Query: (1, 517)


Perhitungan Cosine Similarity

In [42]:
similarity_scores = cosine_similarity(query_vector, tfidf_matrix)
similarity_scores = similarity_scores.flatten()

Ranking

In [43]:
data['Similarity_Score'] = similarity_scores
ranked_results = data.sort_values(by='Similarity_Score', ascending=False)
ranked_results.reset_index(drop=True, inplace=True)

Tampilkan 5 Dokumen Paling Relevan

In [44]:
print("\n✅ Hasil Pemeringkatan (Ranking) Dokumen Paling Relevan:")
print("=" * 60)
display(ranked_results[['No Putusan', 'Lembaga Peradilan', 'Similarity_Score', 'Gabungan_Teks']].head())


✅ Hasil Pemeringkatan (Ranking) Dokumen Paling Relevan:


,No Putusan,Lembaga Peradilan,Similarity_Score,Gabungan_Teks
0,63/Pid.Sus/2025/PN,PN Banda Aceh,0.221687,7 tujuh bungkusan plastik warna bening yang di...
1,62/Pid.Sus/2025/PN,PN Banda Aceh,0.210686,7 tujuh bungkusan plastik warna bening yang di...
2,74/Pid.Sus/2025/PN,PN Banda Aceh,0.183249,15 lima belas bungkusan plastik warna bening y...
3,73/Pid.Sus/2025/PN,PN Banda Aceh,0.183070,15 lima belas bungkusan plastik warna bening y...
4,55/Pid.Sus/2025/PN,PN Banda Aceh,0.170334,1 satu bungkusan plastic bening yang didalamny...


Menampilkan statistik hasil

In [45]:
max_score = ranked_results['Similarity_Score'].max()
total_nonzero = (ranked_results['Similarity_Score'] > 0).sum()
print(f"\nSkor Kemiripan Tertinggi: {max_score:.4f}")
print(f"Jumlah Dokumen yang Relevan (Skor > 0): {total_nonzero} dari {len(data)} dokumen")


Skor Kemiripan Tertinggi: 0.2217
Jumlah Dokumen yang Relevan (Skor > 0): 45 dari 50 dokumen
